BDLE 2024

date du document  :  18/09/2024

# TP1 2024 Préparation de données


Préparation de données

**Auteur :  <br>
runlin ZHOU 28717281 <br>
zhile ZHANG 21201131**

## Préparation du TP

Vérifier que des ressources de calcul sont allouées à votre notebook  (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/TP1

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

drive_dir = "/content/drive/MyDrive/ens/bdle/TP1"
os.makedirs(drive_dir, exist_ok=True)
os.listdir(drive_dir)

Mounted at /content/drive


[]

Commande permettant d'afficher le resultat d'une requete sous la forme d'un tableau interactif

In [ ]:
!pip -q install itables

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.4 MB/s eta 0:00:00


In [ ]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
# affichage du résultat d'une requête

# pour spark
def display(query, n=30):
  return query.limit(n).toPandas()

# pour duckDB
# def display(query, n=30):
#     return query.limit(n).df()


# seulement dans colab (non préconisé en TP):
# from google.colab import data_table
# pour duckDB
# def display(query, n=100):
#     return data_table.DataTable(query.limit(n).df(), include_index=False, num_rows_per_page=10)
# pour spark
# def display(df, n=100):
#   return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

# def display2(df, n=20):
#   pd.set_option('max_columns', None)
#   pd.set_option('max_colwidth', None)
#   return df.limit(n).toPandas()

Installer pyspark et findspark (l'installation dure environ 1 minute):


In [ ]:
!pip install -q pyspark
!pip install -q findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Démarrer la session spark

In [ ]:
import os
import glob

pyspark_dir = glob.glob('/usr/local/lib/python*/dist-packages/pyspark')[0]
print("pyspark directory is", pyspark_dir)
os.environ["SPARK_HOME"] = pyspark_dir
os.environ["JAVA_HOME"] = "/usr"

pyspark directory is /usr/local/lib/python3.10/dist-packages/pyspark


In [ ]:
#!find /usr/local/lib/python3.10/dist-packages/pyspark -name "*netty*"


In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# pour les dataframe et udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")

  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")

  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1727289454436


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [ ]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + "\n"
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    derniere = None
    est_requete = False
    for r in tabRequetes:
        r = r.strip()
        if len(r) > 2:
          derniere = spark.sql(r)
          est_requete = ( r.lower().startswith('select')or r.lower().startswith('with'))
    if(est_requete):
      return display(derniere)
    else:
      return print('ok')

In [ ]:
# facultatif (non préconisé en TP)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

### URL pour l'accès aux datasets

In [ ]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/LqD9N23kxrfHopr"
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/LqD9N23kxrfHopr


### Données de mobilité

Données issues du dataset YFCC

In [ ]:
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

In [ ]:
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

# user visits
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15"
download_file(web_dir, local_dir, "userVisits-Toro.csv")

# poi
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15"
download_file(web_dir, local_dir, "POI-Toro.csv")



os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15/userVisits-Toro.csv save in : /local/data/userVisits-Toro.csv
downloading from URL:  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15/POI-Toro.csv save in : /local/data/POI-Toro.csv


['POI-Toro.csv', 'userVisits-Toro.csv']

### Les visites

Lire les 2 premières lignes du  fichier csv en python.
Est ce que le fichier a une ligne d'entête ?
Quel caractère délimite deux valeurs consécutives dans une ligne de données ?

In [ ]:
f = open(local_dir + "/" + "userVisits-Toro.csv", "r")
print(f.readline()); print(f.readline())

"photoID";"userID";"dateTaken";"poiID";"poiTheme";"poiFreq";"seqID"

7941504100;"10007579@N00";1346844688;30;"Structure";1538;1



Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [ ]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

schema = StructType([
    StructField("photoID", StringType(), True),
    StructField("userID", StringType(), True),
    StructField("dateTaken", StringType(), True),
    StructField("poiID", IntegerType(), True),
    StructField("poiTheme", StringType(), True),
    StructField("poiFreq", IntegerType(), True),
    StructField("seqID", IntegerType(), True)
])

user_visits = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "userVisits-Toro.csv", schema = schema)
user_visits.persist()
user_visits.createOrReplaceTempView("user_visits")
user_visits.show(10)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
|4885404441|10012675@N05|1142732373|    6| Cultural|    986|    2|
|4886008334|10012675@N05|1142732445|    6| Cultural|    986|    2|
|4886009150|10012675@N05|1142916492|    6| Cultural|    986|    3|
|7054481539|10012675@N05|1319327174|   13| Cultural|    964|    4|
|6908387594|10012675@N05|1319328255|   13| Cultural|    964|    4|
|6908381912|10012675@N05|1319331463|   13| Cultural|    964|    4|
|6908398496|10012675@N05|1319331886|   13| Cultural|    964|    4|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 10 rows

root
 |-- photoID: string (nullable 

Vérifier qu'il n'y a pas de date nulle

In [ ]:
%%sql
select count(1) as nb_dates_nulles
from user_visits
where dateTaken is NULL

-- autre solution avec "case"
--select count(case when ..... then date end) as nb_date_nulles
--from user_visits

nb_dates_nulles
Loading ITables v2.2.1 from the internet... (need help?)



Vérifier qu'il n'y a aucune séquence associée à plusieurs utilisateurs

In [ ]:
%%sql
select seqID
from user_visits
group by seqID
having count(poiID) >= 300

seqID
Loading ITables v2.2.1 from the internet... (need help?)



#### POI_sequence

La table **POI_sequence**(seqID, poiID). Les POI visités durant une séquence.
Pour simplifier on compte chaque POI une seule fois par séquence et on ne considère pas l'ordre de visite des POI.

Rmq: il n'est pas nécessaire de préciser le userID pour identifier une séquence.

In [ ]:
%%sql
create or replace temp view POI_sequence as
select seqID, poiID
from user_visits;

select * from POI_sequence

seqID  poiID
0       1     30
1       2      6
2       2      6
3       2      6
4       2      6
5       3      6
6       4     13
7       4     13
8       4     13
9       4     13
10      4     13
11      4     13
12      4     13
13      4     13
14      4     13
15      5     24
16      5     24
17      5     24
18      6     23
19      6     23
20      6     23
21      6     23
22      6     23
23      6     23
24      6     23
25      6     23
26      6     23
27      6     23
28      6     23
29      6     23

### Les lieux visités : POI
Ils sont appelés *Point Of Interest*

In [ ]:
f = open(local_dir + "/" + "POI-Toro.csv", "r")
print(f.readline()); print(f.readline())

poiID;poiName;lat;long;theme

1;Air_Canada_Centre;43.64333; -79.37917;Sport



In [ ]:
poi_schema = "poiID INT, poiName STRING, lat FLOAT, long FLOAT, theme STRING"

poi = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "POI-Toro.csv", schema = poi_schema)
poi.show(3)
poi.printSchema()
poi.createOrReplaceTempView("POI")

+-----+------------------+--------+---------+-----+
|poiID|           poiName|     lat|     long|theme|
+-----+------------------+--------+---------+-----+
|    1| Air_Canada_Centre|43.64333|-79.37917|Sport|
|    2|         BMO_Field|43.63278|-79.41861|Sport|
|    3|Maple_Leaf_Gardens|43.66222|-79.38028|Sport|
+-----+------------------+--------+---------+-----+
only showing top 3 rows

root
 |-- poiID: integer (nullable = true)
 |-- poiName: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- long: float (nullable = true)
 |-- theme: string (nullable = true)



In [ ]:
%%sql
cache table POI;

SELECT *
FROM POI

poiID                        poiName        lat       long      theme
0       1              Air_Canada_Centre  43.643330 -79.379173      Sport
1       2                      BMO_Field  43.632778 -79.418610      Sport
2       3             Maple_Leaf_Gardens  43.662220 -79.380280      Sport
3       4                  Rogers_Centre  43.641392 -79.389168      Sport
4       5             Woodbine_Racetrack  43.712524 -79.602043      Sport
5       6         Art_Gallery_of_Ontario  43.653889 -79.392776   Cultural
6       7            Hockey_Hall_of_Fame  43.646976 -79.377251   Cultural
7       8  Ripley%27s_Aquarium_of_Canada  43.642483 -79.386047   Cultural
8       9         Ontario_Science_Centre  43.716671 -79.338333   Cultural
9      10                 Riverdale_Farm  43.667110 -79.361298   Cultural
10     11           Royal_Ontario_Museum  43.667500 -79.394173   Cultural
11     12                    Toronto_Zoo  43.820278 -79.182777   Cultural
12     13                      Casa_Loma  43.678101 -79.409500   Cultural
13     14   Canadian_National_Exhibition  43.633331 -79.416672  Amusement
14     15     Centreville_Amusement_Park  43.620308 -79.373680  Amusement
15     16                       CN_Tower  43.642601 -79.387100  Amusement
16     17               Exhibition_Place  43.632778 -79.416107  Amusement
17     18       Sunnyside_Amusement_Park  43.637924 -79.450783  Amusement
18     19                Toronto_Islands  43.620831 -79.378609      Beach
19     20                      High_Park  43.646820 -79.462997      Beach
20     21            Yonge-Dundas_Square  43.656109 -79.380280      Beach
21     22         Nathan_Phillips_Square  43.652500 -79.383614      Beach
22     23           Toronto_Eaton_Centre  43.653980 -79.380318   Shopping
23     24            Harbourfront_Centre  43.638489 -79.382065   Shopping
24     25              Kensington_Market  43.654774 -79.400681   Shopping
25     26       Yorkdale_Shopping_Centre  43.725597 -79.452698   Shopping
26     27             Yorkville,_Toronto  43.670280 -79.391113   Shopping
27     28        Old_City_Hall_(Toronto)  43.652500 -79.381668  Structure
28     29   Ontario_Legislative_Building  43.662449 -79.391708  Structure
29     30        Union_Station_(Toronto)  43.645279 -79.380562  Structure

la liste des thèmes

In [ ]:
%%sql
create or replace temp view themes as
select distinct theme
from POI
order by theme;

cache table themes;

select *
from themes

theme
Loading ITables v2.2.1 from the internet... (need help?)


## Exercice 1

#### 1) Les 10 POI les plus photographiés
indications: on peut compléter une requête SQL avec *LIMIT N* pour n'obtenir que N tuples du résultat. Pour une requête dont le résultat est trié (order by) on obtient les N premiers tuples, sinon on obtient N tuples quelconques du résultat.

In [ ]:
%%sql
select  poiID, count(poiID) as nbPhoto
from user_visits
group by poiID
order by nbPhoto desc
limit 10


poiID  nbPhoto
0     11     4139
1     22     3603
2     21     3591
3     16     3553
4      1     3506
5      4     3056
6      7     2053
7     23     1866
8      8     1736
9     25     1701

#### 2) Les visites avec date détaillée
Définir la table Visite_date(userID, seqID, poiID, date, annee, mois, jour, heure) contenant les visites avec la date détaillée composée des attributs : année, mois, jour, heure.
Indication : la date est initialement au format "unix". Voir la fonction de conversion de date *from_unixtime* et la fonction *extract(... from ...)*.

In [ ]:
%%sql
create or replace temp view Visite_Date as
select userID, seqID,
from_unixtime(dateTaken) as date,
extract(year FROM date) as annee,
extract(month FROM date) as mois,
extract(day FROM date) as jour,
extract(hour FROM date) as heure,
extract(minute FROM date) as minute,
cast(extract(second FROM date) as int) as seconde
from user_visits
order by seqID, date;

select * from Visite_Date

userID  seqID                 date  annee  mois  jour  heure  \
0   10007579@N00      1  2012-09-05 11:31:28   2012     9     5     11   
1   10012675@N05      2  2006-03-19 01:30:48   2006     3    19      1   
2   10012675@N05      2  2006-03-19 01:37:28   2006     3    19      1   
3   10012675@N05      2  2006-03-19 01:39:33   2006     3    19      1   
4   10012675@N05      2  2006-03-19 01:40:45   2006     3    19      1   
5   10012675@N05      3  2006-03-21 04:48:12   2006     3    21      4   
6   10012675@N05      4  2011-10-22 23:46:14   2011    10    22     23   
7   10012675@N05      4  2011-10-23 00:04:15   2011    10    23      0   
8   10012675@N05      4  2011-10-23 00:57:43   2011    10    23      0   
9   10012675@N05      4  2011-10-23 01:04:46   2011    10    23      1   
10  10012675@N05      4  2011-10-23 01:06:25   2011    10    23      1   
11  10012675@N05      4  2011-10-23 01:10:00   2011    10    23      1   
12  10012675@N05      4  2011-10-23 01:16:49   2011    10    23      1   
13  10012675@N05      4  2011-10-23 01:19:38   2011    10    23      1   
14  10012675@N05      4  2011-10-23 01:20:48   2011    10    23      1   
15  10014440@N06      5  2007-11-27 01:57:01   2007    11    27      1   
16  10014440@N06      5  2007-11-27 01:58:57   2007    11    27      1   
17  10014440@N06      5  2007-11-27 02:01:18   2007    11    27      2   
18  10014440@N06      6  2007-12-19 08:09:27   2007    12    19      8   
19  10014440@N06      6  2007-12-19 08:10:39   2007    12    19      8   
20  10014440@N06      6  2007-12-19 08:15:42   2007    12    19      8   
21  10014440@N06      6  2007-12-19 08:15:57   2007    12    19      8   
22  10014440@N06      6  2007-12-19 08:16:47   2007    12    19      8   
23  10014440@N06      6  2007-12-19 08:17:24   2007    12    19      8   
24  10014440@N06      6  2007-12-19 08:19:14   2007    12    19      8   
25  10014440@N06      6  2007-12-19 08:21:44   2007    12    19      8   
26  10014440@N06      6  2007-12-19 08:22:06   2007    12    19      8   
27  10014440@N06      6  2007-12-19 08:22:18   2007    12    19      8   
28  10014440@N06      6  2007-12-19 08:25:54   2007    12    19      8   
29  10014440@N06      6  2007-12-19 08:27:12   2007    12    19      8   

    minute  seconde  
0       31       28  
1       30       48  
2       37       28  
3       39       33  
4       40       45  
5       48       12  
6       46       14  
7        4       15  
8       57       43  
9        4       46  
10       6       25  
11      10        0  
12      16       49  
13      19       38  
14      20       48  
15      57        1  
16      58       57  
17       1       18  
18       9       27  
19      10       39  
20      15       42  
21      15       57  
22      16       47  
23      17       24  
24      19       14  
25      21       44  
26      22        6  
27      22       18  
28      25       54  
29      27       12

#### 3a) Le nombre de POI par utilisateur
Le résultat (userID, nbPOI) est trié par nombre décroissant de POI.

In [ ]:
%%sql
SELECT userID, COUNT(DISTINCT poiID) AS nbPOI
FROM user_visits
GROUP BY userID
ORDER BY nbPOI DESC

userID  nbPOI
0      34211328@N00     24
1      43139087@N00     23
2      84987970@N00     22
3      30624156@N00     20
4      63677124@N07     19
5      20741443@N00     19
6      48784629@N00     17
7      24854893@N00     16
8      59525924@N05     15
9      11191102@N07     15
10     71401076@N00     14
11  34427466731@N01     14
12     78436447@N00     14
13     39460517@N03     14
14     37663331@N00     13
15      9449875@N03     13
16     13644648@N03     13
17     23987663@N00     13
18  36521964195@N01     13
19     42536354@N00     13
20     46769540@N00     13
21     76323119@N00     13
22      7776449@N06     13
23      9985167@N04     13
24     37517876@N07     13
25     40437944@N00     13
26     45742513@N00     13
27     91032493@N00     12
28     99771506@N00     12
29     11090433@N05     12

#### 3b) Le nombre de POI par séquence
Définir la table Seq3plus(seqID, nbPoi) correspondant aux séquences qui contiennent au moins 3 POI distincts.
Afficher le résultat trié par nombre de POI décroissant puis numéro de séquence croissant.


In [ ]:
%%sql
create or replace temp view Seq3plus as
select seqID, COUNT(DISTINCT poiID) as nbPOI
from user_visits
GROUP BY seqID
;

select * from seq3plus
order by nbPOI desc, seqID;

seqID  nbPOI
0     298     13
1    4961     10
2    4351      9
3    5964      9
4     510      8
5     525      8
6    5369      8
7    5981      8
8     454      7
9     681      7
10    686      7
11    905      7
12   2072      7
13   2252      7
14   3052      7
15   5254      7
16   5263      7
17   1281      6
18   1358      6
19   1786      6
20   3504      6
21   3636      6
22   4493      6
23   5250      6
24   5276      6
25   5277      6
26    549      5
27    708      5
28    729      5
29    783      5

#### 4) Trajectoire



a) Définir une UDF qui prend en entrée une liste de couples (date, poi) et qui retourne une liste de PoI triés par date croissante. Le resultat ne contient pas 2 PoI consécutifs identiques.

In [ ]:
def trierPOI(listeCouples: List[Tuple[int, int]]) -> List[int]:
  if listeCouples is None:
    return []
  listeCouples.sort(key=lambda x: x[0])
  result = []
  for i, (_, poi) in enumerate(listeCouples):
    if i == 0 or poi != listeCouples[i-1][1]:  # Comparer avec le PoI précédent
      result.append(poi)
  return result

spark.udf.register("trierPOI", trierPOI, ArrayType(IntegerType()))

# test local
print(trierPOI([(13,0),(10,2), (12,1), (14,2),(11,2)]))

[2, 1, 0, 2]


Définir la vue Trajectoire(userID, seqID, listPOI).

listPOI est la liste des POI visités, pendant la séquence seqID, dans l'ordre chronologique

Indications:
pour la sequence 687 la listPOI est [7,16,4,8,4,16]

In [ ]:
%%sql
create or replace temp view Trajectoire as
select userID, seqID, trierPOI(collect_set((dateTaken, poiID))) as listePOI
from user_visits
group by userID, seqID
;

cache table Trajectoire;

--exemple
select * from Trajectoire
where size(listePOI) = 6
order by seqID

userID  seqID                  listePOI
0      14617133@N05    384  [22, 28, 22, 28, 22, 28]
1      20456447@N03    687      [7, 16, 4, 8, 4, 16]
2      20741443@N00    729   [23, 21, 22, 7, 28, 23]
3      27168489@N00   1358   [16, 8, 28, 22, 23, 21]
4      29352917@N00   1524  [29, 30, 16, 29, 16, 29]
5      30624156@N00   1598   [28, 7, 30, 28, 22, 27]
6      32827327@N03   1786   [20, 6, 25, 11, 30, 24]
7      33547369@N00   1877  [23, 21, 23, 22, 23, 22]
8      34128007@N04   1912     [8, 16, 4, 1, 29, 16]
9      34314322@N00   1993  [30, 22, 28, 29, 30, 16]
10  35034347485@N01   2069   [6, 21, 23, 28, 22, 28]
11  35237095252@N01   2158    [30, 7, 30, 16, 4, 16]
12     44925192@N00   2916    [21, 7, 30, 16, 4, 16]
13  49503002894@N01   3401     [22, 8, 22, 8, 22, 3]
14  51035610516@N01   3501    [21, 30, 8, 16, 4, 16]
15  51035681908@N01   3504   [21, 22, 4, 16, 25, 11]
16     84987970@N00   5180  [22, 28, 22, 23, 21, 22]
17     84987970@N00   5250   [30, 23, 21, 22, 6, 16]
18     84987970@N00   5276     [6, 4, 30, 7, 28, 21]
19     86292040@N00   5377     [30, 1, 8, 16, 8, 23]

#### 5) Transitions
5a) Définir la table Transitions(poi1, poi2, nbTrans) avec nbTrans étant le nombre de déplacements directs de poi1 à poi2.

Deux POI apparaissant successivement ($poi_1$ suivi de $poi_2$) dans une séquence forment un déplacement direct entre $poi_1$ et $poi_2$.
Une séquence contenant un seul POI ne correspond à aucun déplacement.

Indication1: définir une UDF transition(listePoi) qui retourne une liste de couples de PoIs ($p_i$, $p_j$).


In [ ]:
def transition(listePOI):
    result = []
    for i in range(1, len(listePOI)):
        el = (listePOI[i-1], listePOI[i])
        result.append(el)
    return result

spark.udf.register("transition",
                   transition,
                   ArrayType(StructType([
                       StructField("p1", IntegerType()),
                       StructField("p2", IntegerType())
                   ])))

# test local
print(transition([2,6,1,6]))

[(2, 6), (6, 1), (1, 6)]


In [ ]:
%%sql
create or replace temp view Transition1 as
select seqID, explode(transition(listePOI)) as transition
from Trajectoire
where size(listePOI)>1
order by seqID
;

select t.seqID, t.transition.p1,  t.transition.p2
from Transition1 t

seqID  p1  p2
0       8  23  24
1      33  21  25
2      58   7  11
3      58  11  27
4      59  11  27
5      67  28  23
6      67  23  22
7      67  22  28
8      71  22  28
9      71  28  23
10     71  23  21
11     75  25   6
12     99   3  23
13     99  23  27
14    100  21  23
15    100  23  30
16    102  23  30
17    103  30   7
18    103   7  30
19    104  23  30
20    105  30  21
21    111  30   1
22    113  30   7
23    113   7  30
24    139   3  23
25    139  23  22
26    139  22  30
27    139  30  22
28    149   2  17
29    155  16   4

In [ ]:
%%sql
create or replace temp view Transition2 as
select t.transition.p1 as poi1, t.transition.p2 as poi2, count(*) as nbtransition
from Transition1 t
group by t.transition.p1 , t.transition.p2;


select * from Transition2
order by nbtransition desc;

poi1  poi2  nbtransition
0     23    21            84
1     22    28            69
2     28    22            66
3     21    23            60
4      7    30            56
5     30     7            51
6     28    23            44
7      8    16            41
8     16     4            40
9     16     8            39
10    23    28            39
11     4    16            35
12    23    22            28
13    22    23            26
14    11    27            25
15    30    16            24
16    22    21            23
17    21    28            21
18    21    22            20
19    27    11            19
20     7    28            19
21    23    30            18
22    28    21            18
23    30    22            17
24     8    30            17
25     1    30            15
26    16    30            15
27     7    21            14
28    16    24            13
29    30     8            13

b) Définir la vue

Transition_relative(poi1, poi2, p)

avec p étant un taux de transition relatif au nombre total de transitions partant d'un POI.

Indication: vous pouvez commencer par créer une vue

Total_transition(poi, t)

avec t étant le nombre total de transitions par poi.

In [ ]:
%%sql
create or replace temp view Total_transition as
select poi1, sum(nbtransition) as t
from Transition2
group by poi1
order by poi1;

select * from Total_transition

poi1    t
0      1   44
1      2   18
2      3   24
3      4   60
4      6   61
5      7  152
6      8   93
7      9    1
8     10    4
9     11   67
10    13   20
11    14   16
12    15    8
13    16  167
14    17   11
15    19   13
16    20    2
17    21  163
18    22  210
19    23  212
20    24   41
21    25   41
22    26    2
23    27   37
24    28  169
25    29   41
26    30  164

In [ ]:
%%sql
create or replace temp view Transition_relative as
select t_t.poi1, t.poi2, t.nbtransition/t_t.t as rel
from Total_transition t_t join Transition2 t on t_t.poi1 = t.poi1
order by poi1, poi2;

select * from Transition_relative
order by  rel desc

poi1  poi2       rel
0      9    11  1.000000
1     14    17  0.812500
2     10    21  0.750000
3     17    14  0.727273
4      2    17  0.666667
5     15    19  0.625000
6      4    16  0.583333
7     27    11  0.513514
8      3    21  0.500000
9     20     6  0.500000
10    20    22  0.500000
11    26    22  0.500000
12    26    21  0.500000
13     8    16  0.440860
14    23    21  0.396226
15    28    22  0.390533
16    11    27  0.373134
17     7    30  0.368421
18    21    23  0.368098
19     1    30  0.340909
20    22    28  0.328571
21    30     7  0.310976
22    19    15  0.307692
23    13    11  0.300000
24     2    14  0.277778
25    28    23  0.260355
26    10     1  0.250000
27    16     4  0.239521
28    16     8  0.233533
29    19     7  0.230769

### 6) DuréeVisitePOI
La durée moyenne de visite d'un POI.
Indication, pour une série d'événements consécutifs associés à un même POI dans une séquence, la durée de visite est la différence de date entre le 1er et le dernier événement.

In [ ]:
def dureeVisite(liste_couples):
    result = []
    if not liste_couples or len(liste_couples) == 1:
        return result

    # init : slow pointer starts at the first element of the list
    slow = 0
    n = len(liste_couples)

    # 快指针从第一个元素遍历到列表末尾
    # fast pointer traverses from index 1 to the end of the list (len(listecouple)).
    for fast in range(1, n):
        # 如果当前poi不同，计算访问时长
        # if the poiID is different
        if liste_couples[fast][0] != liste_couples[slow][0]:
            # calculate the duration of this visite
            duration = liste_couples[fast - 1][1] - liste_couples[slow][1]
            if duration > 0:
                result.append((liste_couples[slow][0], duration))
            # 更新慢指针到新poi的起始点
            # update the position of pointers
            slow = fast

    # 处理最后一段连续相同poi的情况
    # in case of consecutive identical poi in the last segment
    duration = liste_couples[-1][1] - liste_couples[slow][1]
    if duration > 0:
        result.append((liste_couples[slow][0], duration))

    return result



spark.udf.register("dureeVisite", dureeVisite, ArrayType(
                                                StructType(
                                                    [StructField("poiID", IntegerType()),
                                                     StructField("duree", IntegerType())])))

print(dureeVisite([(102, 8), (102, 11), (106, 14), (101, 15), (101, 17), (102, 20), (102, 21), (102, 24)]))


[(102, 3), (101, 2), (102, 4)]


In [ ]:
%%sql
create or replace temp view DureeVisitePOI as
select
    userID,
    seqID,
    explode(dureeVisite(collect_list((poiID,cast(dateTaken as Integer))))) as POI_duree
from user_visits
group by seqID, userID;


select *
from DureeVisitePOI d
order by userID, seqID

userID  seqID    POI_duree
0    10012675@N05      2     (6, 597)
1    10012675@N05      4   (13, 5674)
2    10014440@N06      5    (24, 257)
3    10014440@N06      6   (23, 1065)
4    10014440@N06      7    (23, 573)
5    10014440@N06      8   (23, 1615)
6    10014440@N06      8   (24, 1359)
7    10014440@N06      9    (24, 296)
8    10014440@N06     10   (25, 7991)
9    10014440@N06     11  (17, 31482)
10   10014440@N06     12     (24, 36)
11   10014440@N06     13    (23, 421)
12   10014440@N06     14    (24, 339)
13   10031363@N00     17      (7, 84)
14  101607720@N06     30     (30, 40)
15   10218635@N06     33   (25, 4049)
16   10282509@N00     35   (11, 1811)
17   10285309@N08     36   (1, 11681)
18   10297518@N00     37   (11, 9032)
19   10316646@N07     41     (6, 135)
20   10488855@N00     56    (28, 364)
21   10502709@N05     57    (7, 6484)
22   10502709@N05     58    (7, 5302)
23   10502709@N05     58    (27, 242)
24   10502709@N05     59    (11, 106)
25   10502709@N05     59   (27, 1354)
26   10502709@N05     60    (7, 3739)
27   10502709@N05     62   (11, 3048)
28   10502709@N05     66   (21, 1111)
29   10502709@N05     67   (28, 7241)

In [ ]:
%%sql
create or replace temp view DureeMoyenneVisitePOI as
select
    POI_duree.poiID as poi,
    AVG(POI_duree.duree) / 60 as duree_visite_moyenne  -- transform the seconds to minutes after calculating the averange
from DureeVisitePOI
group by POI_duree.poiID
order by duree_visite_moyenne desc;

select * from DureeMoyenneVisitePOI;

poi  duree_visite_moyenne
0    17            143.449495
1     4            143.393171
2     9            122.456481
3     1            113.289239
4     8            105.013810
5    12             84.107609
6    16             84.087130
7     2             81.938739
8    11             81.032700
9    10             68.782083
10   24             68.034625
11   15             60.830833
12   13             59.003704
13    3             57.126623
14    6             52.529953
15   18             51.870000
16   21             50.626352
17   22             49.837754
18   25             49.564069
19   23             48.990946
20   26             48.503571
21   30             45.227830
22   29             44.297576
23   27             40.995211
24    7             40.292285
25   28             27.058810
26   14             26.517778
27   19             26.136667
28   20             21.777778

### 7) Durée moyenne de déplacement entre deux POIs.

Définir la vue **DureeDeplacement** (poi1, poi2, duree) calculant la duree moyenne de déplacement entre deux POI consécutifs dans une séquence.



## Exercice 2

Charger les données de Geonames

In [ ]:
import zipfile

#geonames
web_dir = PUBLIC_DATASET + "geonames_ALL"

download_file(web_dir, local_dir, "allCountries.zip")

#unzip
local_file = local_dir + "/" + "allCountries.txt"
if(os.path.isfile(local_file)):
    print(local_file, "is already stored")
else:
  with zipfile.ZipFile(local_dir + "/" + "allCountries.zip", 'r') as zip_ref:
    zip_ref.extractall(local_dir)

os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=geonames_ALL/allCountries.zip save in : /local/data/allCountries.zip


['POI-Toro.csv', 'userVisits-Toro.csv', 'allCountries.zip', 'allCountries.txt']

In [ ]:
geonames =  spark.read.option("header", "False").option("delimiter", "\t").format("csv").load(local_dir + "/" + "allCountries.txt")
geonames.createOrReplaceTempView("geonames")
geonames.show(3)

+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|    _c0|                 _c1|                 _c2|                 _c3|     _c4|    _c5|_c6|_c7|_c8|  _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|          _c17|      _c18|
+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|2986043|  Pic de Font Blanca|  Pic de Font Blanca|Pic de Font Blanc...|42.64991|1.53335|  T| PK| AD| NULL|  00|NULL|NULL|NULL|   0|NULL|2860|Europe/Andorra|2014-11-05|
|2994701|            Roc Mélé|            Roc Mele|Roc Mele,Roc Mele...|42.58765|1.74028|  T| MT| AD|AD,FR|  00|NULL|NULL|NULL|   0|NULL|2803|Europe/Andorra|2014-11-05|
|3007683|Pic des Langounelles|Pic des Langounelles|Pic des Langounelles|42.61203|1.47364|  T| PK| AD|AD,FR|  00|NULL|NULL|NULL|   0|NULL|2685|Europe/Andorr

### 1) Geonames
Définir le schéma, limité au 9 premiers attributs donnant des informations sur l'identifiant d'un POI, ses noms, sa position GPS, le code du pays..., ), d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

#####1a) schéma de Geonames

Définir Geonames2 en précisant le schéma : nom et type des attributs.
Se limiter au 9 premiers attributs donnant des informations sur l'identifiant d'un POI, ses noms, sa position GPS, le code du pays..., ), d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

In [ ]:
%%sql
create or replace temp view Geonames2 as
select cast(_c0 as long) as geonameID,
_c1 as name,
-- _c2 as ascii_name,
_c3 as alternate_name,
cast(_c4 as double) as latitude,
cast(_c5 as double) as longitude,
_c6 as feature_class,
_c7 as feature_code,
_c8 as country_code
from Geonames;

select * from Geonames2

geonameID                          name  \
0     2986043            Pic de Font Blanca   
1     2994701                      Roc Mélé   
2     3007683          Pic des Langounelles   
3     3017832          Pic de les Abelletes   
4     3017833       Estany de les Abelletes   
5     3023203  Port Vieux de la Coume d’Ose   
6     3029315          Port de la Cabanette   
7     3034945                     Port Dret   
8     3038814               Costa de Xurius   
9     3038815               Font de la Xona   
10    3038816                     Xixerella   
11    3038818                       Riu Xic   
12    3038819                   Pas del Xic   
13    3038820                  Roc del Xeig   
14    3038821              Collada del Xeig   
15    3038822                   Fonts Vives   
16    3038823                  Roc de Vista   
17    3038824                Obaga de Vista   
18    3038825                 Coll de Vista   
19    3038826                 Coll de Vista   
20    3038827                     Visanseny   
21    3038828               Roc de la Vinya   
22    3038829             Canal de la Vinya   
23    3038830                Bosc de Villar   
24    3038831               Torrent de Vila   
25    3038832                          Vila   
26    3038833              Basers de Vicenç   
27    3038834                   Pla de Viàs   
28    3038835             Vial del Cardaire   
29    3038836                   Font del Vi   

                                       alternate_name  latitude  longitude  \
0                      Pic de Font Blanca,Pic du Port  42.64991    1.53335   
1                         Roc Mele,Roc Meler,Roc Mélé  42.58765    1.74028   
2                                Pic des Langounelles  42.61203    1.47364   
3   Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...  42.52535    1.73343   
4   Estany de les Abelletes,Etang de Font-Negre,Ét...  42.52915    1.73362   
5   Port Vieux de Coume d'Ose,Port Vieux de Coume ...  42.62568    1.61823   
6      Port de la Cabanette,Porteille de la Cabanette  42.60000    1.73333   
7   Port Dret,Port de Bareites,Port de las Bareyte...  42.60172    1.45562   
8                                                None  42.50692    1.47569   
9                                                None  42.55003    1.44986   
10                                               None  42.55327    1.48736   
11                                               None  42.57165    1.67554   
12                                               None  42.49766    1.57597   
13                                               None  42.56068    1.48980   
14                                               None  42.57512    1.49009   
15                                               None  42.49829    1.56428   
16                                               None  42.49922    1.48118   
17                                               None  42.47935    1.44621   
18                                               None  42.47611    1.60578   
19                                      Coll de Vista  42.47552    1.44173   
20                                Visanceny,Visanseny  42.57488    1.61616   
21                                               None  42.52776    1.56446   
22                                               None  42.52232    1.52319   
23                                               None  42.60135    1.54340   
24                                               None  42.53414    1.56678   
25                                    Casas Vila,Vila  42.53176    1.56654   
26                                               None  42.47669    1.48290   
27                                               None  42.59001    1.65996   
28                                               None  42.56372    1.50503   
29                                               None  42.52342    1.47698   

   feature_class feature_code country_code  
0              T           PK           AD  
1              T           MT        

##### 1b) Extrait pour le Canada

Définir la vue Geonames_canada pour les POI situés au Canada.

In [ ]:
%%sql
create or replace temp view Geonames_canada as
select geonameID, name, alternate_name, latitude, longitude, feature_class, feature_code
from Geonames2
where ...

cache table Geonames_canada;

select * from  Geonames_canada;

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '.'.(line 4, pos 6)

== SQL ==
create or replace temp view Geonames_canada as
select geonameID, name, alternate_name, latitude, longitude, feature_class, feature_code
from Geonames2
where ...
------^^^

cache table Geonames_canada


### 2) Association entre les POI et Geonames
Compléter les POI avec des attributs de geonames (par exemple name et [feature code](http://www.geonames.org/export/codes.html))